# Download MIMIOC Mixed-Layer Depths

We'll download [MIMOC](https://www.pmel.noaa.gov/mimoc/) data here and update variable and coordinate names to be compatible with the other data.

In [ ]:
# parameters
mimoc_url = "https://www.pmel.noaa.gov/mimoc/data/MIMOC_ML_v2.2_PT_S.zip -o mimoc/MIMOC_ML_v2.2_PT_S.zip"
mimoc_mld_file = "tmp_mimoc_mld.nc"

## Download and extract data

In [ ]:
!rm -rfv mimoc/
!mkdir -p mimoc
!curl {mimoc_url}
!cd mimoc && unzip MIMOC_ML_v2.2_PT_S.zip

## Extract mixed layer depth and fix labels

In [ ]:
import xarray as xr
import numpy as np
from pathlib import Path

In [ ]:
mimoc_files = sorted(Path("mimoc").glob("*.nc"))
mimoc_files

In [ ]:
ds = xr.concat([xr.open_dataset(mf) for mf in mimoc_files], dim="month")
ds

In [ ]:
mld = ds["DEPTH_MIXED_LAYER"].rename("mixed_layer_depth")
mld = mld.rename({"LAT": "lat", "LONG": "lon"})
mld = mld.transpose("month", "lat", "lon")
mld.coords["lat"] = xr.DataArray(ds["LATITUDE"].isel(month=0).data, dims=("lat", ))
mld.coords["lon"] = xr.DataArray(ds["LONGITUDE"].isel(month=0).data, dims=("lon", ))
mld.coords["month"] = xr.DataArray(np.arange(1, 13), dims=("month", ))
mld.attrs["units"] = "m"

## Have a look

In [ ]:
mld.sel(lat=slice(-35, 35)).plot.contourf(col="month", col_wrap=3);

## Save

In [ ]:
mld.to_dataset().to_netcdf(mimoc_mld_file)